In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy
import seaborn as sns
import numpy as np

In [2]:
coordinates = pd.read_csv("/Volumes/TwoTeras/Resources/Agent_Coordinates_Exp2_Summary.csv")

In [3]:
coordinates.head()

,ped_id,x_coord_ped,y_coord_ped
0,56_Sa,-166.452744,131.991852
1,39_Sa,-194.456741,-20.814148
2,19_Cma,8.543274,-6.224136
3,55_Sa,-192.756729,-167.554153
4,25_Cma,-139.216721,-48.324142


In [4]:
def approach_window(data):
    """
    Given the data of the participant, get a dataframe that contains information about 
    the pedestrians that the participant interacted with and the time window for each 
    of these interactions.

    Parameters:
        data (DataFrame): The data of the participant.

    Returns:
        DataFrame: A dataframe containing pedestrian ID and the time window of interaction 
                   (gaze onsets on faces).
    """
    # Get the first timestamp in the data
    first_timestamp = data['date_seconds'].min()

    # Get the gazes onset
    gaze_onset = data[data["events"] == 2.0]
    
    # Get gazes onset on Agents
    face_gaze_static_individual_ped = gaze_onset[gaze_onset['Collider_CategoricalN'].str.contains(r'Face', regex=True, na=False)]
    
    # Get each name for encountered pedestrian
    names = face_gaze_static_individual_ped["names"].tolist()

    # Check if there were no interaction of the participant with static pedestrians
    if len(face_gaze_static_individual_ped) == 0:
        # Fill the columns of the dataframe with NaN values
        data = {
            'ped_id': [np.nan],
            'w_l': [np.nan],
            'w_u': [np.nan]
        }
        df_sorted = pd.DataFrame(data)
    
    else:
        # Define the windows (50 seconds before and 150 seconds after)
        window_lower = face_gaze_static_individual_ped['date_seconds'] - pd.Timedelta(seconds=50)
        window_upper = face_gaze_static_individual_ped['date_seconds'] + pd.Timedelta(seconds=150)

        # Check if any window_lower is earlier than the first timestamp in the data
        window_lower = window_lower.apply(lambda x: max(x, first_timestamp))

        # Construct a dataframe with all the important information extracted
        d = {
            "ped_name": names,
            "window lower": window_lower.tolist(),
            "window upper": window_upper.tolist()
        }
        df_summary = pd.DataFrame(d)

        # Detect consecutive duplicates by creating a group indicator for consecutive pedestrian encounters
        df_summary['t_v'] = (df_summary['ped_name'].shift() != df_summary['ped_name']).cumsum()

        # Create an empty list to store the group results
        group_results = []

        # Look at each group of the same pedestrian (t_v indicates consecutive duplicates)
        for i, group in df_summary.groupby('t_v'):
            # Check if the time difference between consecutive duplicates is less than 50 seconds
            group['same_range'] = (group['window lower'].diff() < pd.Timedelta(seconds=50)).eq(False).cumsum()

            # Aggregate by the new grouping (same_range) to merge consecutive duplicates
            agg_dict = {
                # First pedestrian name in the group
                "ped_id": pd.NamedAgg(column='ped_name', aggfunc='first'),
                # Minimum 'window lower' in the group
                "w_l": pd.NamedAgg(column='window lower', aggfunc='min'),
                # Maximum 'window upper' in the group
                "w_u": pd.NamedAgg(column='window upper', aggfunc='max')
            }

            # Aggregate the results for each group and append to the list
            result = group.groupby('same_range').agg(**agg_dict)
            group_results.append(result)

        # Concatenate the results to create the final sorted dataframe
        df_sorted = pd.concat(group_results).reset_index(drop=True)

    # Return the resulting dataframe
    return df_sorted



In [5]:
def calculate_minimal_distances(df_sorted, time, data):
    """
    This function calculates the minimal distances between the participant and the pedestrians they interacted with,
    and returns a DataFrame containing the pedestrian ID, minimal distance, and the number of interactions.

    Parameters:
        df_sorted (DataFrame): Contains information about the participant and the pedestrians they interacted with.
        time (list): The time data for the participant.
        data (DataFrame): The position data of the participant.

    Returns:
        DataFrame: Contains 'ped_id', 'minimal_distance', and 'num_interactions' for each interaction.
    """
    minimal_distance = []
    ped_ids = []
    num_interactions = []

    # If no interactions occurred
    if pd.isna(df_sorted['ped_id'].iloc[0]):
        return pd.DataFrame({'ped_id': [np.nan], 'minimal_distance': [np.nan], 'num_interactions': [0]})

    # Retrieve key details from the sorted dataframe
    names = df_sorted.ped_id.tolist()
    window_lower = df_sorted.w_l.tolist()
    window_upper = df_sorted.w_u.tolist()
    x_ped_list = df_sorted.x_coord_ped.tolist()
    y_ped_list = df_sorted.y_coord_ped.tolist()

    for i in range(len(window_lower)):
        ped_id = names[i]
        w_low = window_lower[i]
        w_up = window_upper[i]
        x_ped = x_ped_list[i]
        y_ped = y_ped_list[i]

        # Get time window data for this pedestrian
        ts = time[time.index(list(filter(lambda j: j > w_low, time))[0]):time.index(list(filter(lambda j: j < w_up, time))[-1]) + 1]
        data_small = data.iloc[time.index(ts[0]):(time.index(ts[-1]) + 1)]

        # Get participant's position at each time point and adjust relative to pedestrian
        x_coordinate_par = data_small["playerBodyPosition.x"]
        y_coordinate_par = data_small["playerBodyPosition.z"]
        x = [i - x_ped for i in x_coordinate_par]
        y = [j - y_ped for j in y_coordinate_par]

        # Calculate Euclidean distance
        distances = np.sqrt(np.array(x)**2 + np.array(y)**2)

        # Filter out distances greater than 13
        filtered_distances = distances[distances < 13]

        # If there are valid distances, calculate the minimal distance
        if len(filtered_distances) > 0:
            min_distance = np.nanmin(filtered_distances)

            # Only consider distances less than 10 as valid interactions
            if min_distance < 10:
                minimal_distance.append(min_distance)
                ped_ids.append(ped_id)
                num_interactions.append(1)
    
    # Create a DataFrame with the results
    result_df = pd.DataFrame({
        'ped_id': ped_ids,
        'minimal_distance': minimal_distance,
        'num_interactions': num_interactions,
        
    })
    
    return result_df


In [6]:
import pandas as pd
import glob

# Your path
path = "/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze" 

# Load all CSV files in the path
files = glob.glob(path + "/*.csv")

# Create a list to store failed files and their errors
failed_files = []

# Loop through all files
for filename in files:
    try:
        # Read the participant data
        One_participant = pd.read_csv(filename)
        One_participant = One_participant.loc[:, One_participant.columns[One_participant.columns.get_loc('SubjectID'):]]

        # Convert the 'timeStampDataPointEnd' to datetime, specifying it's in seconds since epoch
        One_participant['date_seconds'] = pd.to_datetime(One_participant['timeStampDataPointEnd'], unit='s')
        time = One_participant['date_seconds'].tolist()

        # Apply the window function to get all agent interactions
        Windows = approach_window(One_participant)

        # Check if Windows is empty and handle accordingly
        if Windows.empty:
            print(f"No interactions for {filename[-10:-4]}, skipping.")
            continue  # Skip to the next file if no interactions

        print(filename[-10:-4])

        # Convert 'ped_id' in both DataFrames to strings before merging
        Windows['ped_id'] = Windows['ped_id'].astype(str)
        coordinates['ped_id'] = coordinates['ped_id'].astype(str)

        # Perform a left merge on 'ped_id' to add 'x_coord_ped' and 'y_coord_ped' to Windows
        merged_df = Windows.merge(coordinates, on='ped_id', how='left')

        # Save the merged data to a CSV
        merged_df.to_csv(f"/Volumes/TwoTeras/1_Experiment_2/Proxemics/Windows_Faces/{filename[-10:-4]}.csv", index=True)

        # Calculate Minimal Distance Participant-wise
        Minimal_Distance = calculate_minimal_distances(merged_df, time, One_participant)
        Minimal_Distance["SubjectID"] = filename[-10:-6]
        Minimal_Distance["Session"] = filename[-5]

        # Save the minimal distance data to a CSV
        Minimal_Distance.to_csv(f"/Volumes/TwoTeras/1_Experiment_2/Proxemics/Minimal_Distance_Faces/{filename[-10:-4]}.csv", index=True)

    except Exception as e:
        # If an error occurs, add the filename and the error message to the list
        failed_files.append({'filename': filename[-10:-4], 'error': str(e)})

# After the loop, create a DataFrame from the failed files list
if failed_files:
    failed_df = pd.DataFrame(failed_files)

    # Save the DataFrame to a CSV file for reference
    failed_df.to_csv("/Volumes/TwoTeras/1_Experiment_2/Proxemics/failed_files_log.csv", index=False)

    print(f"Failures logged for {len(failed_files)} files.")
else:
    print("No failures detected.")


1031_1
1031_2
1031_3
1031_4
1031_5
1268_1
1268_2
1268_3
1268_4
1268_5
1574_1
1574_2
1574_3
1574_4
1574_5
1843_1
1843_2
1843_3
1843_4
1843_5
2069_1
2069_2
2069_3
2069_4
2069_5
3193_1
3193_2
3193_3
3193_4
3193_5
3540_1
3540_2
3540_3
3540_4
3540_5
4580_1
4580_2
4580_3
4580_4
4580_5
4598_1
4598_2
4598_3
4598_4
4598_5
4847_1
4847_2
4847_3
4847_4
4847_5
4875_1
4875_2
4875_3
4875_4
4875_5
5161_1
5161_2
5161_3
5161_4
5161_5
5189_1
5189_2
5189_3
5189_4
5189_5
5743_1
5743_2
5743_3
5743_4
5743_5
5766_1
5766_2
5766_3
5766_4
5766_5
5851_1
5851_2
5851_3
5851_4
5851_5
5972_1
5972_2
5972_3
5972_4
5972_5
6406_1
6406_2
6406_3
6406_4
6406_5
7081_1
7081_2
7081_3
7081_4
7081_5
7393_1
7393_2
7393_3
7393_4
7393_5
7823_1
7823_2
7823_3
7823_4
7823_5
7935_1
7935_2
7935_3
7935_4
7935_5
8629_1
8629_2
8629_3
8629_4
8629_5
9297_1
9297_2
9297_3
9297_4
9297_5
9627_1
9627_2
9627_3
9627_4
9627_5
1142_1
1142_2
1142_3
1142_4
1142_5
1234_1
1234_2
1234_3
1234_4
1234_5
6266_1
6266_2
6266_3
6266_4
6266_5
5191_1
5191_2
5191_3

In [7]:
data = pd.read_csv("/Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/0365_1.csv")

In [8]:
data.columns

Index(['Unnamed: 0.2', 'level_0', 'Unnamed: 0', 'index', 'Unnamed: 0.1',
       'SubjectID', 'Session', 'SessionSubsection', 'timeStampDataPointEnd',
       'combinedGazeValidityBitmask', 'eyePositionCombinedWorld.x',
       'eyePositionCombinedWorld.y', 'eyePositionCombinedWorld.z',
       'eyeDirectionCombinedWorld.y', 'eyeDirectionCombinedWorld.z',
       'eyeDirectionCombinedLocal.x', 'eyeDirectionCombinedLocal.y',
       'eyeDirectionCombinedLocal.z', 'playerBodyPosition.x',
       'playerBodyPosition.y', 'playerBodyPosition.z', 'hitColliderType',
       'hitObjectColliderName', 'ordinalOfHit', 'hitPointOnObject_x',
       'hitPointOnObject_y', 'hitPointOnObject_z', 'Eucledian_distance',
       'Collider_Categorical', 'Face_Hits', 'Time_Shift', 'Continuous_Time',
       'Bitmask_flag', 'Interpolated_collider', ' eyePositionCombinedWorld.x',
       'Collider_shift', 'counter', 'Time_of_Gaze', 'Gaze', 'combined_vel',
       'thresh', 'isFix', 'corrected_vel', 'events', 'length', 'di

In [9]:
# Drop all the old indexes 
df = data.loc[:, data.columns[data.columns.get_loc('SubjectID'):]]
# Convert the 'timeStampDataPointEnd' to datetime, specifying it's in seconds since epoch
df['date_seconds'] = pd.to_datetime(df['timeStampDataPointEnd'], unit='s')
df.head()

,SubjectID,Session,SessionSubsection,timeStampDataPointEnd,combinedGazeValidityBitmask,eyePositionCombinedWorld.x,eyePositionCombinedWorld.y,eyePositionCombinedWorld.z,eyeDirectionCombinedWorld.y,eyeDirectionCombinedWorld.z,...,thresh,isFix,corrected_vel,events,length,distance,avg_dist,names,Collider_CategoricalN,date_seconds
0,365,1,1,1.635519e+09,3,-59.082943,2.440343,34.884834,0.043033,0.997212,...,44.460563,NaN,NaN,NaN,NaN,NaN,NaN,Building_161,Building,2021-10-29 14:55:46.468227584
1,365,1,1,1.635519e+09,3,-59.082943,2.440343,34.884834,0.040692,0.997121,...,44.460563,0.000000,NaN,2.0,0.672079,63.996796,63.996938,Building_161,Building,2021-10-29 14:55:46.476162816
2,365,1,1,1.635519e+09,3,-59.082943,2.440343,34.884834,0.040266,0.997135,...,44.460563,0.000000,NaN,NaN,0.672079,63.996796,63.996938,Building_161,Building,2021-10-29 14:55:46.484594432
3,365,1,1,1.635519e+09,3,-59.090160,2.440291,34.884834,0.041908,0.997886,...,44.460563,73.369491,NaN,NaN,0.672079,63.997106,63.996938,Building_161,Building,2021-10-29 14:55:46.493522688
4,365,1,1,1.635519e+09,3,-59.090191,2.440290,34.884834,0.042011,0.997881,...,44.460563,0.642210,0.64221,NaN,0.672079,63.997107,63.996938,Building_161,Building,2021-10-29 14:55:46.504435968


In [10]:
df.Collider_CategoricalN.unique()

array(['Building', 'Background', 'TaskBuilding_Public', 'Global_Landmark',
       'Passive_Agent_Face', 'Passive_Agent', 'TaskBuilding_Residential',
       'Active_Agent', 'Active_Agent_Face'], dtype=object)

In [11]:
#Perform a left merge on 'ped_id' to add 'x_coord_ped' and 'y_coord_ped' to df2
merged_df = Windows.merge(coordinates, on='ped_id', how='left')
merged_df.head()

,ped_id,w_l,w_u,x_coord_ped,y_coord_ped
0,20_Cma,2023-08-24 11:59:45.441822208,2023-08-24 12:02:55.208123136,40.563278,59.675858
1,13_Cma,2023-08-24 12:00:21.321241600,2023-08-24 12:03:41.731929344,29.403275,164.265854
2,08_Cma,2023-08-24 12:01:12.770823680,2023-08-24 12:04:32.770823680,-41.546722,138.255859
3,46_Sa,2023-08-24 12:04:05.862910464,2023-08-24 12:07:25.996334336,96.525269,-195.884140
4,56_Sa,2023-08-24 12:06:49.742790144,2023-08-24 12:10:09.742790144,-166.452744,131.991852


In [12]:
display(Windows.shape)
display(merged_df.shape)

(8, 3)

(8, 5)

In [13]:
# plotting the social and personal space
        circle2 = plt.Circle((0, 0), 1.22, color='#EE7718', edgecolor='#FAB87C', alpha=0.75)
        circle1 = plt.Circle((0, 0), 3.65, color='#FAB87C', alpha=0.55, linewidth=1)
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        # plotting the zero lines in black
        plt.axhline(color='black', lw=0.5)
        plt.axvline(color='black', lw=0.5)
        # labeling the x and y axis
        plt.xlabel("x", size = 20)
        plt.ylabel("z", size = 20)
        # axis limits
        plt.xlim(-9, +9)
        plt.ylim(-9, +9)
        # axis ticks size
        plt.xticks(size=20)
        plt.yticks(size=20)
        # adjusting the padding of the trajectory map plot
        plt.subplots_adjust(right=0.7)
        plt.tight_layout(pad = 1, rect= (0.5,0.5,0.5,0.5))

IndentationError: unexpected indent (363381571.py, line 2)

In [ ]:
""" Given the dataframe that contains all encountered pedestrian for each participant , their location and time window
of interaction, the time data and the data of each participant (df_sorted), draw the trajectory map which depict the
pathway around each pedestrian interacted with, get this pedestrian identification number and calculate the minimum
distance of this pathway, get the number of interactions of the participant with static individual pedestrians.
Parameters: df_sorted which contain information about the participant and the pedestrians they interacted with, the
time data and the data of the participant.
Returns: The number of interaction of the participant with static indivdual pedestrians, a list of the minimum distances
of the different interactions and a list of "names" of the pedestrians the participant interacted with. """
def get_trajectory_map_and_minimal_distance(df_sorted, time, data):
    minimal_distance = []
    names_in = []
    df_for_all_ped = []
    # check if there were interactions with pedestrians
    if pd.isna(df_sorted['ped_id'].iloc[0]):
        # if none the number of interaction is 0
        number_of_interactions = 0
        # Create an empty DataFrame with the same columns and column names
        data = {
            'ped_id': [np.nan],
            'x': [np.nan],
            'y': [np.nan]}

        df_plot_result = pd.DataFrame(data)
        # creating an empty map plot for no interactions
        fig, ax = plt.subplots(1, 1, figsize=(8 ,8), dpi=140)
        # plotting the social and personal space
        circle2 = plt.Circle((0, 0), 1.22, color='#EE7718', edgecolor='#FAB87C', alpha=0.75)
        circle1 = plt.Circle((0, 0), 3.65, color='#FAB87C', alpha=0.55, linewidth=1)
        # zooming in on the map
        ax.set_xlim(-10, 10)
        ax.set_ylim(-10, 10)
        plt.axhline(color='black', lw=0.5)
        plt.axvline(color='black', lw=0.5)
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        plt.subplots_adjust(right=0.95)
        plt.tight_layout(pad=0.8, rect=(0.5, 0.5, 0.5, 0.5))
        minimal_distance.append(np.nan)
        names_in.append(np.nan)

    # if there are interactions
    else:
        # get the list of names of the encountered pedestrians
        names = df_sorted.ped_id.tolist()
        # get the list of lower window values in the time window for each pedestrian
        window_lower = df_sorted.w_l.tolist()
        # get the list of upper window values in the time window for each pedestrian
        window_upper = df_sorted.w_u.tolist()
        # get the list of the x coordinates for each pedestrian
        x_ped_list = df_sorted.x_coord_ped.tolist()
        # get the list of the y coordinates for each pedestrian
        y_ped_list = df_sorted.y_coord_ped.tolist()
        # plot the pathway for each pedestrian accordingly

        # create a figure
        fig, ax = plt.subplots(1, 1, figsize=(8,8), dpi = 140)
        # get the name, location and window range around each pedestrian
        for i in range(len(window_lower)):

            ped_id = names[i]
            w_low = window_lower[i]
            w_up = window_upper[i]
            x_ped = x_ped_list[i]
            y_ped = y_ped_list[i]

            # get the time window data
            ts = time[time.index(list(filter(lambda j: j > w_low, time))[0]):time.index(list(filter(lambda j: j < w_up, time))[-1]) + 1]
            data_small = (data.iloc[time.index(ts[0]):(time.index(ts[-1]) + 1)])
            # get the participant location at each time point
            x_coordinate_par = data_small["playerBodyPosition.x"]
            y_coordinate_par = data_small["playerBodyPosition.z"]
            # get x,y location of participant (taking into account ped location is (0,0) always)
            x = [i - x_ped for i in x_coordinate_par]
            y = [j - y_ped for j in y_coordinate_par]
            # create data frame with the coordinates pathway around each participant
            m = {"ped_id": ped_id, "x": x, "y": y}
            df_plot = pd.DataFrame(m)
            # calculate the Euclidean distance for each pathway around each pedestrian
            distance = ((df_plot["y"]) ** 2 + (df_plot["x"]) ** 2) ** 0.5
            # insert the calculated  distances in the df_plot
            df_plot.insert(column="distance", value=distance, loc=3)
            # filter out any  distance that is greater than 13
            df_plot = df_plot[df_plot.distance < 13]
            # create a list of the filtered distances that remained
            distance_df = df_plot.distance.tolist()
            # get the list of the pedestrian names
            name_1 = df_plot["ped_id"].tolist()

            # check if the list of distances not empty
            if len(distance_df) != 0:
                #calculate the minimum distance of the pathway
                min_distance = np.nanmin(distance_df)

                # only consider the minimum distances less than 10 as an interaction
                if min_distance < 10:
                    name_to_insert = name_1[0]
                    names_in.append(name_to_insert)
                    minimal_distance.append(min_distance)

                # list to save the collected data if each pedestrian encountered
                df_for_all_ped.append(df_plot)
                # the resulting dataframe includes all encountered pedestrian data
                df_plot_result = pd.concat(df_for_all_ped)
                # the palette color grey
                palette = sns.color_palette(['gray'])
                # Plotting the trajectory map for each participant
                ax = sns.scatterplot(data=df_plot_result, x='x', y='y', alpha=0.05, hue='ped_id', palette= palette, linewidths = 0.1, legend = False)

        # get the number of interactions for each participant
        number_of_interactions = len(names_in)

        # title and axes organization of the trajectory map plot

        # plotting the social and personal space
        circle2 = plt.Circle((0, 0), 1.22, color='#EE7718', edgecolor='#FAB87C', alpha=0.75)
        circle1 = plt.Circle((0, 0), 3.65, color='#FAB87C', alpha=0.55, linewidth=1)
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        # plotting the zero lines in black
        plt.axhline(color='black', lw=0.5)
        plt.axvline(color='black', lw=0.5)
        # labeling the x and y axis
        plt.xlabel("x", size = 20)
        plt.ylabel("z", size = 20)
        # axis limits
        plt.xlim(-9, +9)
        plt.ylim(-9, +9)
        # axis ticks size
        plt.xticks(size=20)
        plt.yticks(size=20)
        # adjusting the padding of the trajectory map plot
        plt.subplots_adjust(right=0.7)
        plt.tight_layout(pad = 1, rect= (0.5,0.5,0.5,0.5))
        # showing the map
       # plt.show()

    return number_of_interactions,minimal_distance, names_in

In [ ]:
def get_trajectory_map_and_minimal_distance(df_sorted, time, data):
    """
    Given the dataframe that contains all encountered pedestrians for each participant, their location, 
    and time window of interaction, this function draws the trajectory map depicting the pathway around 
    each pedestrian, gets the pedestrian ID, and calculates the minimum distance of this pathway.
    
    Parameters:
        df_sorted (DataFrame): Contains information about participants and the pedestrians they interacted with.
        time (list): The time data for the participant.
        data (DataFrame): The position data of the participant.
    
    Returns:
        tuple: Number of interactions, list of minimum distances, and list of pedestrian names interacted with.
    """
    minimal_distance = []
    names_in = []
    df_for_all_ped = []

    # If no interactions occurred
    if pd.isna(df_sorted['ped_id'].iloc[0]):
        number_of_interactions = 0
        data_empty = {
            'ped_id': [np.nan],
            'x': [np.nan],
            'y': [np.nan]
        }
        df_plot_result = pd.DataFrame(data_empty)
        minimal_distance.append(np.nan)
        names_in.append(np.nan)
    else:
        # Retrieve key details from the sorted dataframe
        names = df_sorted.ped_id.tolist()
        window_lower = df_sorted.w_l.tolist()
        window_upper = df_sorted.w_u.tolist()
        x_ped_list = df_sorted.x_coord_ped.tolist()
        y_ped_list = df_sorted.y_coord_ped.tolist()

        fig, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=140)

        for i in range(len(window_lower)):
            ped_id = names[i]
            w_low = window_lower[i]
            w_up = window_upper[i]
            x_ped = x_ped_list[i]
            y_ped = y_ped_list[i]

            # Get time window data for this pedestrian
            ts = time[time.index(list(filter(lambda j: j > w_low, time))[0]):time.index(list(filter(lambda j: j < w_up, time))[-1]) + 1]
            data_small = data.iloc[time.index(ts[0]):(time.index(ts[-1]) + 1)]

            # Get participant's position for each time point and adjust relative to pedestrian
            x_coordinate_par = data_small["playerBodyPosition.x"]
            y_coordinate_par = data_small["playerBodyPosition.z"]
            x = [i - x_ped for i in x_coordinate_par]
            y = [j - y_ped for j in y_coordinate_par]

            # Create a dataframe for plotting and distance calculations
            df_plot = pd.DataFrame({"ped_id": ped_id, "x": x, "y": y})
            df_plot['distance'] = np.sqrt(df_plot['x']**2 + df_plot['y']**2)

            # Filter out distances greater than 13
            df_plot = df_plot[df_plot['distance'] < 13]
            distance_df = df_plot['distance'].tolist()

            if distance_df:
                min_distance = np.nanmin(distance_df)

                # Only consider distances less than 10 as valid interactions
                if min_distance < 10:
                    names_in.append(ped_id)
                    minimal_distance.append(min_distance)

                # Save the data for plotting
                df_for_all_ped.append(df_plot)
                df_plot_result = pd.concat(df_for_all_ped, ignore_index=True)

        # Plot the trajectory if there are valid interactions
        if not df_plot_result.empty:
            sns.scatterplot(data=df_plot_result, x='x', y='y', alpha=0.05, hue='ped_id', palette=['gray'], linewidth=0.1, legend=False)

        number_of_interactions = len(names_in)

    # Plot the social and personal space
    circle2 = plt.Circle((0, 0), 1.22, color='#EE7718', edgecolor='#FAB87C', alpha=0.75)
    circle1 = plt.Circle((0, 0), 3.65, color='#FAB87C', alpha=0.55, linewidth=1)
    ax.add_patch(circle1)
    ax.add_patch(circle2)

    # Plot axis settings
    plt.axhline(color='black', lw=0.5)
    plt.axvline(color='black', lw=0.5)
    plt.xlabel("x", size=20)
    plt.ylabel("z", size=20)
    plt.xlim(-9, 9)
    plt.ylim(-9, 9)
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.subplots_adjust(right=0.7)
    plt.tight_layout(pad=1, rect=(0.5, 0.5, 0.5, 0.5))

    return number_of_interactions, minimal_distance, names_in

In [ ]:
calculate_minimal_distances(merged_df, time, df)

In [ ]:
time = df['date_seconds'].to_list()
get_trajectory_map_and_minimal_distance(merged_df, time, df)

In [ ]:
def get_trajectory_map_and_minimal_distance_category(df_sorted, time, data):
    """
    Given the dataframe that contains all encountered pedestrians for each participant, their location, 
    and time window of interaction, this function draws separate trajectory maps for 'Cma' and 'Sa' pedestrians, 
    calculates the minimum distance of this pathway, and returns interaction details.
    
    Parameters:
        df_sorted (DataFrame): Contains information about participants and the pedestrians they interacted with.
        time (list): The time data for the participant.
        data (DataFrame): The position data of the participant.
    
    Returns:
        tuple: Number of interactions, list of minimum distances, and list of pedestrian names interacted with.
    """
    minimal_distance = []
    names_in = []
    df_for_all_ped_cma = []
    df_for_all_ped_sa = []

    # If no interactions occurred
    if pd.isna(df_sorted['ped_id'].iloc[0]):
        number_of_interactions = 0
        minimal_distance.append(np.nan)
        names_in.append(np.nan)
    else:
        # Retrieve key details from the sorted dataframe
        names = df_sorted.ped_id.tolist()
        window_lower = df_sorted.w_l.tolist()
        window_upper = df_sorted.w_u.tolist()
        x_ped_list = df_sorted.x_coord_ped.tolist()
        y_ped_list = df_sorted.y_coord_ped.tolist()

        for i in range(len(window_lower)):
            ped_id = names[i]
            w_low = window_lower[i]
            w_up = window_upper[i]
            x_ped = x_ped_list[i]
            y_ped = y_ped_list[i]

            # Get time window data for this pedestrian
            ts = time[time.index(list(filter(lambda j: j > w_low, time))[0]):time.index(list(filter(lambda j: j < w_up, time))[-1]) + 1]
            data_small = data.iloc[time.index(ts[0]):(time.index(ts[-1]) + 1)]

            # Get participant's position for each time point and adjust relative to pedestrian
            x_coordinate_par = data_small["playerBodyPosition.x"]
            y_coordinate_par = data_small["playerBodyPosition.z"]
            x = [i - x_ped for i in x_coordinate_par]
            y = [j - y_ped for j in y_coordinate_par]

            # Create a dataframe for plotting and distance calculations
            df_plot = pd.DataFrame({"ped_id": ped_id, "x": x, "y": y})
            df_plot['distance'] = np.sqrt(df_plot['x']**2 + df_plot['y']**2)

            # Filter out distances greater than 13
            df_plot = df_plot[df_plot['distance'] < 13]
            distance_df = df_plot['distance'].tolist()

            if distance_df:
                min_distance = np.nanmin(distance_df)

                # Only consider distances less than 10 as valid interactions
                if min_distance < 10:
                    names_in.append(ped_id)
                    minimal_distance.append(min_distance)

                # Separate into 'Sa' and 'Cma' groups
                if ped_id.endswith('_Sa'):
                    df_for_all_ped_sa.append(df_plot)
                elif ped_id.endswith('_Cma'):
                    df_for_all_ped_cma.append(df_plot)

    # Plot side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), dpi=140)

    # Plot for 'Sa' pedestrians
    if df_for_all_ped_sa:
        df_plot_sa = pd.concat(df_for_all_ped_sa, ignore_index=True)
        sns.scatterplot(data=df_plot_sa, x='x', y='y', alpha=0.05, hue='ped_id', palette=['gray'], linewidth=0.1, legend=False, ax=ax1)
        ax1.set_title("Trajectory for Passive Agents")
        circle1 = plt.Circle((0, 0), 1.22, color='#FF6347', edgecolor='#FAB87C', alpha=0.75)  # Center circle in 'tomato' for 'Sa'
        circle2 = plt.Circle((0, 0), 3.65, color='#FFA07A', alpha=0.55, linewidth=1)         # Larger circle in 'light salmon' for 'Sa'
        ax1.add_patch(circle1)
        ax1.add_patch(circle2)
        ax1.axhline(color='black', lw=0.5)
        ax1.axvline(color='black', lw=0.5)
        ax1.set_xlim(-9, 9)
        ax1.set_ylim(-9, 9)
        ax1.set_xlabel("x", size=20)
        ax1.set_ylabel("z", size=20)
        ax1.tick_params(axis='both', labelsize=15)

    # Plot for 'Cma' pedestrians
    if df_for_all_ped_cma:
        df_plot_cma = pd.concat(df_for_all_ped_cma, ignore_index=True)
        sns.scatterplot(data=df_plot_cma, x='x', y='y', alpha=0.05, hue='ped_id', palette=['gray'], linewidth=0.1, legend=False, ax=ax2)
        ax2.set_title("Trajectory for Active Agents")
        circle1 = plt.Circle((0, 0), 1.22, color='#1E90FF', edgecolor='#87CEEB', alpha=0.75)  # Center circle in 'dodgerblue' for 'Cma'
        circle2 = plt.Circle((0, 0), 3.65, color='#B0E0E6', alpha=0.55, linewidth=1)         # Larger circle in 'powderblue' for 'Cma'
        ax2.add_patch(circle1)
        ax2.add_patch(circle2)
        ax2.axhline(color='black', lw=0.5)
        ax2.axvline(color='black', lw=0.5)
        ax2.set_xlim(-9, 9)
        ax2.set_ylim(-9, 9)
        ax2.set_xlabel("x", size=20)
        ax2.set_ylabel("z", size=20)
        ax2.tick_params(axis='both', labelsize=15)

    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4)

    number_of_interactions = len(names_in)
    return number_of_interactions, minimal_distance, names_in


In [ ]:
time = df['date_seconds'].to_list()
get_trajectory_map_and_minimal_distance_category(merged_df, time, df)